In [ ]:
!pip install transformers
!pip install simpletransformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


English Data

### Mixing english and tamil translated to english data

In [3]:
import pandas as pd
concatenated=pd.read_csv('/content/drive/MyDrive/NLP-Project/BalencedLGBT_dataset.csv')
concatenated['category'] = concatenated['category'].replace('Non-anti-LGBT+ content', 'ally')
concatenated['category'] = concatenated['category'].replace('ally', 0)
concatenated['category'] = concatenated['category'].replace('Homophobic', 1)
concatenated['category'] = concatenated['category'].replace('Transphobic', 2)

In [4]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

df = concatenated

X = df['text']
y = df['category']

testsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

train_val_idx, test_idx = next(testsplit.split(X, y))

X_train_val, X_test = X.iloc[train_val_idx], X.iloc[test_idx]
y_train_val, y_test = y.iloc[train_val_idx], y.iloc[test_idx]


validsplit = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)

train_idx, val_idx = next(validsplit.split(X_train_val, y_train_val))

X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
y_train, y_val = y_train_val.iloc[train_idx], y_train_val.iloc[val_idx]

train_df = pd.DataFrame({'text': X_train, 'category': y_train})
val_df = pd.DataFrame({'text': X_val, 'category': y_val})
test_df = pd.DataFrame({'text': X_test, 'category': y_test})

print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')
print(f'Test set shape: {test_df.shape}')


Training set shape: (5331, 2)
Validation set shape: (1777, 2)
Test set shape: (1778, 2)


In [5]:
print(train_df['category'].value_counts())
print(val_df['category'].value_counts())
print(test_df['category'].value_counts())

1    1795
0    1794
2    1742
Name: category, dtype: int64
0    598
1    598
2    581
Name: category, dtype: int64
0    599
1    598
2    581
Name: category, dtype: int64


In [6]:
train_df=train_df[["text", "category"]]
train_df = train_df.rename(columns={'text': 'text', 'category': 'labels'})
val_df=val_df[["text", "category"]]
val_df = val_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})

In [7]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:

import emoji
train_df['text'] = train_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
val_df['text'] = val_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

## Training Model with Englis Data

In [9]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# define hyperparameter
train_args ={"reprocess_input_data": True,
             "overwrite_output_dir": True,
             "fp16":False,
             "num_train_epochs": 3,
             "max_seq_length": 128,
             "train_batch_size": 16,
             "eval_batch_size": 16,
             "logging_steps": 50,
             "save_steps": 2000,
             "learning_rate": 3e-5}

# Create a ClassificationModel
model = ClassificationModel(
    "xlmroberta", "xlm-roberta-base",
    num_labels=3,
    args=train_args
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [10]:
# Train the model
model.train_model(train_df)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 0 of 3:   0%|          | 0/334 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/334 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/334 [00:00<?, ?it/s]

(1002, 0.3843971796067963)

Validation Set


In [11]:
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='macro')
result, model_outputs, wrong_predictions = model.eval_model(val_df, f1=f1_multiclass, acc=accuracy_score)
result

  0%|          | 0/1777 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/112 [00:00<?, ?it/s]

{'mcc': 0.8877700180953702,
 'f1': 0.9254728995525651,
 'acc': 0.9251547552054024,
 'eval_loss': 0.2686389034242893}

Testing With English Data

In [12]:
import pandas as pd
e_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
e_test = e_test.rename(columns={'text                        ': 'text'})
#e_test=e_test[["text", "category"]] 
e_test = e_test.dropna()
test_df = e_test
test_df['category'] = test_df['category'].replace('Non-anti-LGBT+ content', 'ally')
test_df['category'] = test_df['category'].replace('ally', 0)
test_df['category'] = test_df['category'].replace('Homophobic', 1)
test_df['category'] = test_df['category'].replace('Transphobic', 2)
test_df=test_df[["text", "category"]]
test_df = test_df.rename(columns={'text': 'text', 'category': 'labels'})
test_df['text'] = test_df['text'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))

In [13]:
test = test_df['text'].tolist()
labels = test_df['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"For those who speak about culture., pre marita...",0,1
1,Royal Queen You must know that laws are also ...,0,1
2,The person who asking completely doesn't know...,0,1
3,we all defenetly know that those drinks are no...,0,1
4,This is even among animals and many other spec...,0,1
5,Don't play there emissions idiots ...,2,0
6,Most girls harassment are done by heterosexual...,0,1
7,we're the railway POLICE OH I'M SORRY THEY SLE...,1,0
8,They are not male or female..they are some peo...,1,0
9,Avoid to make this type of videos . Days' soon...,1,0


In [14]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.99      0.95      0.97       924
  homophobic       0.54      0.85      0.66        61
 transphobic       0.67      0.80      0.73         5

    accuracy                           0.94       990
   macro avg       0.73      0.87      0.78       990
weighted avg       0.96      0.94      0.95       990



### Checking Tamil data performance

In [15]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_tam_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'tamil'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['tamil'] = combined_data['tamil'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,tamil,ln
0,0,Spr....2016 poitan feel happy with my partner ...,வசந்தம்....2016 போனேன் என் பார்ட்னருடன் மகிழ்ச...,eng_tam
1,0,R u still with ur partner,நீங்கள் இன்னும் உங்கள் துணையுடன் இருக்கிறீர்கள்,eng_tam
2,0,excellent movie..no unnecessary drama or scene...,அருமையான திரைப்படம்..தேவையற்ற நாடகம் அல்லது கா...,eng_tam
3,0,"For those who speak about culture., pre marita...","கலாச்சாரத்தைப் பற்றி பேசுபவர்களுக்கு, திருமணத்...",eng_tam
4,0,Best movie and people not understand relations...,சிறந்த திரைப்படம் மற்றும் மக்கள் உறவைப் புரிந்...,eng_tam
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,வருண் தனது தாயின் முக்கிய அன்பையும் நிழலையும் ...,eng_tam
986,0,i am really crying pro crying_face crying_fac...,நான் உண்மையிலேயே அழுகிறேன் ப்ரோ அழுகை_முகம் அழ...,eng_tam
987,0,They may be transgender but don't ever forgot ...,அவர்கள் திருநங்கைகளாக இருக்கலாம் ஆனால் அவர்களு...,eng_tam
988,0,It is their own choice. I support them No doub...,அது அவர்களின் சொந்த விருப்பம். நான் அவர்களை ஆத...,eng_tam


In [16]:

test = combined_data['eng'].tolist()
org = combined_data['tamil'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['tamil_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['tamil_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['tamil_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.9191919191919192


,category,eng,tamil,ln,eng_pred,tamil_pred
30,0,Section 377 thumbs_up thumbs_up ...,பிரிவு 377 தம்ஸ் அப் தம்ஸ் அப்,eng_tam,0,1
36,0,Can you share your thoughts on cannabis and th...,கஞ்சா மற்றும் THC 'tetrahydrocannabinol' கலவை ...,eng_tam,0,1
54,0,Royal Queen You must know that laws are also ...,அரச ராணி ஆண்களால் சட்டங்கள் உருவாக்கப்படுகின்...,eng_tam,1,0
60,1,Both of u who replied to my comment thanks for...,எனது கருத்துக்கு பதிலளித்த நீங்கள் இருவரும் இத...,eng_tam,1,0
62,0,The person who asking completely doesn't know...,என்று கேட்கிறவனுக்கு ஓரினச்சேர்க்கை பற்றி முழு...,eng_tam,1,0
...,...,...,...,...,...,...
908,2,"I'm sharing my experience pls read it, two tra...",நான் என் அனுபவத்தை பகிர்ந்து கொள்கிறேன் தயவு ச...,eng_tam,2,0
940,0,exactly omg im not even gay but those answers ...,"நான் ஓரினச்சேர்க்கையாளர் கூட இல்லை, ஆனால் அவர்...",eng_tam,0,1
952,0,No comments... simply waste... ...,கருத்துகள் இல்லை... வெறுமனே வீணடிக்க...,eng_tam,0,1
962,0,Solute for everybady.....,அனைவருக்கும் தீர்வு.....,eng_tam,1,0


In [17]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['tamil_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.97      0.93      0.95       924
  homophobic       0.34      0.56      0.42        61
 transphobic       0.00      0.00      0.00         5

    accuracy                           0.90       990
   macro avg       0.44      0.49      0.46       990
weighted avg       0.92      0.90      0.91       990



In [18]:
test = combined_data['tamil'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"கலாச்சாரத்தைப் பற்றி பேசுபவர்களுக்கு, திருமணத்...",0,1
1,பிரிவு 377 தம்ஸ் அப் தம்ஸ் அப்,0,1
2,கஞ்சா மற்றும் THC 'tetrahydrocannabinol' கலவை ...,0,1
3,எனது கருத்துக்கு பதிலளித்த நீங்கள் இருவரும் இத...,1,0
4,இந்திய சிவில் சமூகம் 377 தீ தீ தீ,0,1
...,...,...,...
94,"நான் ஓரினச்சேர்க்கையாளர் கூட இல்லை, ஆனால் அவர்...",0,1
95,கருத்துகள் இல்லை... வெறுமனே வீணடிக்க...,0,1
96,ஆனால் இந்து மத சாஸ்திரங்களின்படி அது இல்லை.,0,1
97,அனீஷ் கூல் 70 ஆண்டுகளில் உருவாக்கப்பட்ட ஒரு சட...,0,1


# Training with english and testing with Hindi

In [19]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_hin_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'hindi'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['hindi'] = combined_data['hindi'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,hindi,ln
0,0,Spr....2016 poitan feel happy with my partner ...,Spr....2016 कवि अपने पार्टनर के साथ खुश हैं,eng_hin
1,0,R u still with ur partner,क्या आप अभी भी अपने साथी के साथ हैं,eng_hin
2,0,excellent movie..no unnecessary drama or scene...,उत्कृष्ट फिल्म..कोई अनावश्यक नाटक या दृश्य नही...,eng_hin
3,0,"For those who speak about culture., pre marita...","संस्कृति की बात करने वालों के लिए, शादी से पहल...",eng_hin
4,0,Best movie and people not understand relations...,बेहतरीन फिल्म और लोग रिश्ते को नहीं समझते हैं ...,eng_hin
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,ऐसा लगता है कि कार्तिक ने वरुण पर फायदा उठाया ...,eng_hin
986,0,i am really crying pro crying_face crying_fac...,मैं सच में रो रहा हूँ चेहरा रोना_चेहरा रोना_चे...,eng_hin
987,0,They may be transgender but don't ever forgot ...,वो ट्रांसजेंडर हो सकते हैं लेकिन ये कभी नहीं भ...,eng_hin
988,0,It is their own choice. I support them No doub...,यह उनकी अपनी पसंद है। मैं उनका समर्थन करता हूं...,eng_hin


In [20]:

test = combined_data['eng'].tolist()
org = combined_data['hindi'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['hindi_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['hindi_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['hindi_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.9242424242424242


,category,eng,hindi,ln,eng_pred,hindi_pred
6,0,99% tamilans cant understand r accept it becau...,99% इटालियंस इसे समझ या स्वीकार नहीं कर सकते क...,eng_hin,0,1
30,0,Section 377 thumbs_up thumbs_up ...,धारा 377 थम्स अप थम्स अप,eng_hin,0,1
41,0,it's purely biological !,यह विशुद्ध रूप से जैविक है!,eng_hin,0,1
54,0,Royal Queen You must know that laws are also ...,रॉयल क्वीन आपको पता होना चाहिए कि कानून भी पु...,eng_hin,1,0
60,1,Both of u who replied to my comment thanks for...,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",eng_hin,1,0
...,...,...,...,...,...,...
946,0,pensive_face pensive_face pensive_face pen...,सख़्त_चेहरा सख़्त_चेहरा सख़्त_चेहरा सख़्त_चेहर...,eng_hin,0,2
962,0,Solute for everybady.....,सभी के लिए सल्यूट.....,eng_hin,1,0
964,0,Hats off bro advance 1millon subscriber vara ...,हैट्स ऑफ ब्रो एडवांस 1 मिलियन सब्सक्राइबर वारा...,eng_hin,0,2
980,0,4 arivu athiham bro... Arivey illathavan kudut...,4 अरिवु अथिहम ब्रो... अरिवे इल्लतावन कुदुथा थी...,eng_hin,0,2


In [21]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['hindi_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.97      0.93      0.95       924
  homophobic       0.40      0.61      0.48        61
 transphobic       0.10      0.20      0.13         5

    accuracy                           0.91       990
   macro avg       0.49      0.58      0.52       990
weighted avg       0.93      0.91      0.92       990



In [22]:
test = combined_data['hindi'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"संस्कृति की बात करने वालों के लिए, शादी से पहल...",0,1
1,99% इटालियंस इसे समझ या स्वीकार नहीं कर सकते क...,0,1
2,धारा 377 थम्स अप थम्स अप,0,1
3,यह विशुद्ध रूप से जैविक है!,0,1
4,"आप दोनों जिन्होंने मेरी टिप्पणी का जवाब दिया, ...",1,0
...,...,...,...
86,सख़्त_चेहरा सख़्त_चेहरा सख़्त_चेहरा सख़्त_चेहर...,0,2
87,लेकिन हिन्दू शास्त्रों के अनुसार ऐसा नहीं है।,0,1
88,हैट्स ऑफ ब्रो एडवांस 1 मिलियन सब्सक्राइबर वारा...,0,2
89,अनीश कूल एक कानून जो सिर्फ 70 साल की अवधि में ...,0,1


# Training with english and testing with Telugu

In [23]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_tel_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'tel'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['tel'] = combined_data['tel'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,tel,ln
0,0,Spr....2016 poitan feel happy with my partner ...,వసంతం....2016 పోయిటన్ నా భాగస్వామితో సంతోషంగా ...,eng_tel
1,0,R u still with ur partner,మీరు ఇప్పటికీ మీ భాగస్వామితో ఉన్నారు,eng_tel
2,0,excellent movie..no unnecessary drama or scene...,అద్భుతమైన సినిమా..అనవసరమైన డ్రామా లేదా సన్నివే...,eng_tel
3,0,"For those who speak about culture., pre marita...","సంస్కృతి గురించి మాట్లాడే వారికి, వివాహానికి మ...",eng_tel
4,0,Best movie and people not understand relations...,ఉత్తమ చిత్రం మరియు వ్యక్తులు సంబంధాన్ని అర్థం ...,eng_tel
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,కార్తీక్ వరుణ్ తన తల్లి నుండి తన ప్రధాన ప్రేమన...,eng_tel
986,0,i am really crying pro crying_face crying_fac...,నేను నిజంగా ఏడుస్తున్నాను ప్రో క్రయింగ్_ఫేస్ క...,eng_tel
987,0,They may be transgender but don't ever forgot ...,వారు లింగమార్పిడి కావచ్చు కానీ వారు కూడా మనుషు...,eng_tel
988,0,It is their own choice. I support them No doub...,అది వారి స్వంత ఎంపిక. నేను వారికి మద్దతు ఇస్తు...,eng_tel


In [24]:

test = combined_data['eng'].tolist()
org = combined_data['tel'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['tel_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['tel_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['tel_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.898989898989899


,category,eng,tel,ln,eng_pred,tel_pred
6,0,99% tamilans cant understand r accept it becau...,99% ఇటాలియన్లు దీనిని అర్థం చేసుకోలేరు లేదా అం...,eng_tel,0,1
9,0,Ak Be The Change I don't think it's good ..it...,అక్ బీ ది చేంజ్ ఇది మంచిది కాదని నేను అనుకుంట...,eng_tel,0,1
16,0,stop explaining her as transgender is there an...,లింగమార్పిడి అని ఆమెను వివరించడం మానేయండి,eng_tel,0,1
33,0,Cool Bans yaaa.. I will accept.. even I done...,కూల్ బీన్స్ యా,eng_tel,0,2
36,0,Can you share your thoughts on cannabis and th...,మీరు గంజాయి మరియు THC 'tetrahydrocannabinol సమ...,eng_tel,0,1
...,...,...,...,...,...,...
924,0,Sheik Dawood Where's the concept of a wife in ...,షేక్ దావూద్ స్వలింగ సంపర్కంలో భార్య అనే భావన ఎ...,eng_tel,0,1
942,0,India doesn't consider gay sex as a crime any ...,భారతదేశం స్వలింగ సంపర్కాన్ని నేరంగా పరిగణించదు...,eng_tel,0,1
962,0,Solute for everybady.....,అందరికీ పరిష్కారం.....,eng_tel,1,0
980,0,4 arivu athiham bro... Arivey illathavan kudut...,4 అరివు అతిహమ్ బ్రో... అరివే ఇల్లతవన్ కుడుత తీ...,eng_tel,0,1


In [25]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['tel_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.98      0.89      0.93       924
  homophobic       0.30      0.70      0.43        61
 transphobic       0.00      0.00      0.00         5

    accuracy                           0.87       990
   macro avg       0.43      0.53      0.45       990
weighted avg       0.93      0.87      0.90       990



In [26]:
test = combined_data['tel'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"సంస్కృతి గురించి మాట్లాడే వారికి, వివాహానికి మ...",0,1
1,99% ఇటాలియన్లు దీనిని అర్థం చేసుకోలేరు లేదా అం...,0,1
2,అక్ బీ ది చేంజ్ ఇది మంచిది కాదని నేను అనుకుంట...,0,1
3,లింగమార్పిడి అని ఆమెను వివరించడం మానేయండి,0,1
4,కూల్ బీన్స్ యా,0,2
...,...,...,...
119,భారతదేశం స్వలింగ సంపర్కాన్ని నేరంగా పరిగణించదు...,0,1
120,కానీ హిందూ గ్రంధాల ప్రకారం అది కాదు.,0,1
121,అనీష్ కూల్ కేవలం 70 ఏళ్లలో రూపొందించిన చట్టం క...,0,1
122,4 అరివు అతిహమ్ బ్రో... అరివే ఇల్లతవన్ కుడుత తీ...,0,1


# Training with english and testing with Malyalam

In [27]:
df_test=pd.read_csv('/content/eng_3_test.tsv', delimiter='\t')
df_test = df_test.rename(columns={'text                        ': 'eng'}) 
df_test = df_test.dropna()

o_test=pd.read_csv('/content/eng_ml_test.tsv', delimiter='\t') 
o_test = o_test.rename(columns={'text': 'mal'}) 
o_test = o_test.dropna()

combined_data = pd.concat([df_test, o_test], axis=1)
combined_data = combined_data.loc[:, ~combined_data.columns.str.contains('^Unnamed')]
combined_data = combined_data.T.drop_duplicates().T
combined_data
combined_data['category'] = combined_data['category'].replace('Non-anti-LGBT+ content', 'ally')
combined_data['category'] = combined_data['category'].replace('ally', 0)
combined_data['category'] = combined_data['category'].replace('Homophobic', 1)
combined_data['category'] = combined_data['category'].replace('Transphobic', 2)
combined_data['eng'] = combined_data['eng'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data['mal'] = combined_data['mal'].apply(lambda x: emoji.demojize(x, delimiters=(" ", " ")))
combined_data

,category,eng,mal,ln
0,0,Spr....2016 poitan feel happy with my partner ...,Spr....2016 പോയതാൻ എന്റെ പങ്കാളിയുമായി സന്തോഷമ...,eng_ml
1,0,R u still with ur partner,നിങ്ങൾ ഇപ്പോഴും നിങ്ങളുടെ പങ്കാളിക്കൊപ്പമാണ്,eng_ml
2,0,excellent movie..no unnecessary drama or scene...,മികച്ച സിനിമ.. അനാവശ്യ നാടകങ്ങളോ രംഗങ്ങളോ ഇല്ല...,eng_ml
3,0,"For those who speak about culture., pre marita...","സംസ്കാരത്തെക്കുറിച്ച് സംസാരിക്കുന്നവർക്ക്, വിവ...",eng_ml
4,0,Best movie and people not understand relations...,മികച്ച സിനിമയും ആളുകൾക്ക് ബന്ധം മനസ്സിലാകാത്തത...,eng_ml
...,...,...,...,...
985,0,Looks like Karthik took advantage on Varun whe...,അമ്മയിൽ നിന്നുള്ള പ്രധാന സ്നേഹവും നിഴലുകളും നഷ...,eng_ml
986,0,i am really crying pro crying_face crying_fac...,ഞാൻ ശരിക്കും കരയുന്നു പ്രോ കരയുന്നു_മുഖം കരയുന...,eng_ml
987,0,They may be transgender but don't ever forgot ...,അവർ ട്രാൻസ്‌ജെൻഡർ ആയിരിക്കാം പക്ഷെ അവരും മനുഷ്...,eng_ml
988,0,It is their own choice. I support them No doub...,അത് അവരുടെ സ്വന്തം തിരഞ്ഞെടുപ്പാണ്. ഞാൻ അവരെ പ...,eng_ml


In [28]:

test = combined_data['eng'].tolist()
org = combined_data['mal'].tolist()
labels=combined_data['category'] .tolist()

combined_data['eng_pred'], _ = model.predict(test)
combined_data['mal_pred'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['eng_pred'], combined_data['mal_pred'])

print("similarity in predection score:",sim)
combined_data[combined_data['eng_pred'] != combined_data['mal_pred']]


  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.9161616161616162


,category,eng,mal,ln,eng_pred,mal_pred
50,0,shinchan shinchan Don't live by a book. Even ...,ഷിഞ്ചൻ ഷിഞ്ചൻ ഒരു പുസ്തകത്താൽ ജീവിക്കരുത്. നി...,eng_ml,0,1
54,0,Royal Queen You must know that laws are also ...,രാജകീയ രാജ്ഞി നിയമങ്ങൾ നിർമ്മിക്കുന്നത് പുരുഷ...,eng_ml,1,0
57,0,Bro everyone have they own right even lisbean ...,"ബ്രോ എല്ലാവർക്കും അവരുടെ സ്വന്തം അവകാശമുണ്ട്, ...",eng_ml,0,1
60,1,Both of u who replied to my comment thanks for...,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,eng_ml,1,0
63,0,Indian civil society 377 fire fire fire ...,ഇന്ത്യൻ സിവിൽ സൊസൈറ്റി 377 തീ തീ,eng_ml,0,2
...,...,...,...,...,...,...
924,0,Sheik Dawood Where's the concept of a wife in ...,ഷേക്ക് ദാവൂദ് സ്വവർഗ്ഗാനുരാഗത്തിൽ ഭാര്യ എന്ന ആ...,eng_ml,0,1
962,0,Solute for everybady.....,എല്ലാവർക്കും പരിഹാരം.....,eng_ml,1,0
967,0,Jugal Romil of course me too ...,ജുഗൽ റോമിൽ തീർച്ചയായും ഞാനും,eng_ml,0,2
980,0,4 arivu athiham bro... Arivey illathavan kudut...,4 അറിവ് അതിഹം ബ്രോ... അറിവ് ഇല്ലതവൻ കുടുത തീർപ...,eng_ml,0,1


In [29]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, combined_data['mal_pred'], target_names=target_names))

              precision    recall  f1-score   support

        ally       0.97      0.91      0.94       924
  homophobic       0.33      0.62      0.43        61
 transphobic       0.12      0.20      0.15         5

    accuracy                           0.89       990
   macro avg       0.48      0.58      0.51       990
weighted avg       0.93      0.89      0.91       990



In [30]:
test = combined_data['mal'].tolist()
labels = combined_data['category'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"സംസ്കാരത്തെക്കുറിച്ച് സംസാരിക്കുന്നവർക്ക്, വിവ...",0,1
1,ഷിഞ്ചൻ ഷിഞ്ചൻ ഒരു പുസ്തകത്താൽ ജീവിക്കരുത്. നി...,0,1
2,"ബ്രോ എല്ലാവർക്കും അവരുടെ സ്വന്തം അവകാശമുണ്ട്, ...",0,1
3,എന്റെ അഭിപ്രായത്തിന് മറുപടി നൽകിയ നിങ്ങൾ രണ്ടു...,1,0
4,ചോദിക്കുന്നയാൾക്ക് സ്വവർഗാനുരാഗികളെ കുറിച്ച് അ...,0,1
...,...,...,...
103,എന്നാൽ ഹിന്ദു ഗ്രന്ഥങ്ങൾ പ്രകാരം അങ്ങനെയല്ല.,0,1
104,"അനീഷ് കൂൾ, 70 വർഷത്തിനുള്ളിൽ സൃഷ്ടിക്കപ്പെട്ട ...",0,1
105,ജുഗൽ റോമിൽ തീർച്ചയായും ഞാനും,0,2
106,4 അറിവ് അതിഹം ബ്രോ... അറിവ് ഇല്ലതവൻ കുടുത തീർപ...,0,1


Here we rephrased few sentences and tested on our model to analysie its performance

Robustness test

## Robustness Analysis

In [31]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Synonymn aug

In [32]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import random

data = test_df.copy()
syn_aug = naw.SynonymAug(aug_src='wordnet')
lis = []
for index, row in data.iterrows():
    text = syn_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,Spr. .. . 2016 poitan feel well chosen with my...
1,R u still with ur partner,0,R u even so with ur married person
2,excellent movie..no unnecessary drama or scene...,0,fantabulous movie. . no unnecessary dramatic p...
3,"For those who speak about culture., pre marita...",0,"For those who speak about culture. , pre marit..."
4,Best movie and people not understand relations...,0,Best movie and people not understand relations...
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on Varun whe...
986,i am really crying pro crying_face crying_fac...,0,i am really rank pro crying_face crying_face c...
987,They may be transgender but don't ever forgot ...,0,They crataegus oxycantha be transgender but do...
988,It is their own choice. I support them No doub...,0,Information technology is their own option. I ...


In [33]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.701010101010101


,text,labels,Changed_text,Changed_text_labels,text_labels
0,Spr....2016 poitan feel happy with my partner ...,0,Spr. .. . 2016 poitan feel well chosen with my...,1,0
6,99% tamilans cant understand r accept it becau...,0,ninety nine% tamilans cant understand r accept...,1,0
7,U r very clean talker... Body for mud... Only ...,0,Uranium r rattling uninfected talker. .. Eubst...,1,0
9,Ak Be The Change I don't think it's good ..it...,0,Ak Be The Alteration I don ' t conceive inform...,1,0
10,Good movie.. All the characters are gone good....,0,Good movie. . All the characters comprise gone...,1,0
...,...,...,...,...,...
981,Her point of you in a right path......... i su...,0,Her gunpoint of you in a right itinerary. .. ....,1,0
983,Hats off handshake .... Semma semma finally t...,0,Hats off shake. .. . Semma semma ultimately sn...,1,0
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on Varun whe...,1,0
987,They may be transgender but don't ever forgot ...,0,They crataegus oxycantha be transgender but do...,2,0


In [34]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Spr. .. . 2016 poitan feel well chosen with my...,0,1
1,"For those who speak about culture. , pre marit...",0,1
2,ninety nine% tamilans cant understand r accept...,0,1
3,Uranium r rattling uninfected talker. .. Eubst...,0,1
4,Ak Be The Alteration I don ' t conceive inform...,0,1
...,...,...,...
322,Worst thirunangai half wit che go way,0,2
323,Hats off shake. .. . Semma semma ultimately sn...,0,1
324,Looks like Karthik took advantage on Varun whe...,0,1
325,They crataegus oxycantha be transgender but do...,0,2


In [35]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.99      0.65      0.79       924
  homophobic       0.17      0.90      0.28        61
 transphobic       0.09      1.00      0.17         5

    accuracy                           0.67       990
   macro avg       0.42      0.85      0.41       990
weighted avg       0.94      0.67      0.75       990



## RandomCharAug

In [36]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import random

data = test_df.copy()
char_aug = nac.RandomCharAug(action="insert")
lis = []
for index, row in data.iterrows():
    text = char_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,Spr. .. . R20c16 pHoictan ffeeql ha0p6py with ...
1,R u still with ur partner,0,R u still wQi%th ur Opa(rtnLer
2,excellent movie..no unnecessary drama or scene...,0,+excellkeDnt movie. . no unnecessary drama or ...
3,"For those who speak about culture., pre marita...",0,"For those who speak @abcout culture. , pre mar..."
4,Best movie and people not understand relations...,0,Best movie and nppeople not understand SrelQat...
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on Varun whe...
986,i am really crying pro crying_face crying_fac...,0,i am really cry+ibng pro crying_face crying_fa...
987,They may be transgender but don't ever forgot ...,0,They may be trapnsTgfeynder but don ' t ever f...
988,It is their own choice. I support them No doub...,0,It is Pthteir own choice. I CsKup)port thYe@m ...


In [37]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.6989898989898989


,text,labels,Changed_text,Changed_text_labels,text_labels
0,Spr....2016 poitan feel happy with my partner ...,0,Spr. .. . R20c16 pHoictan ffeeql ha0p6py with ...,1,0
7,U r very clean talker... Body for mud... Only ...,0,U r vCe+ry cl3ea2n etalBker. .. BPodXy for mud...,1,0
9,Ak Be The Change I don't think it's good ..it...,0,Ak Be The ChbangQe I don ' t FthEink it ' s yg...,1,0
10,Good movie.. All the characters are gone good....,0,G^oo!d movie. . All the characters are gone go...,1,0
21,My point of you it's correct because there all...,0,My pboinlt of you it ' s 0comr&rect be1cyaDuse...,1,0
...,...,...,...,...,...
982,Worst thirunangai idiot che go way ...,0,Wo6rMst thipruwnEanWgai idiot che go way,1,2
983,Hats off handshake .... Semma semma finally t...,0,THaEts off handshake. .. . Semma semma finally...,1,0
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on Varun whe...,1,0
987,They may be transgender but don't ever forgot ...,0,They may be trapnsTgfeynder but don ' t ever f...,1,0


In [38]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Spr. .. . R20c16 pHoictan ffeeql ha0p6py with ...,0,1
1,"For those who speak @abcout culture. , pre mar...",0,1
2,U r vCe+ry cl3ea2n etalBker. .. BPodXy for mud...,0,1
3,Ak Be The ChbangQe I don ' t FthEink it ' s yg...,0,1
4,G^oo!d movie. . All the characters are gone go...,0,1
...,...,...,...
313,Wo6rMst thipruwnEanWgai idiot che go way,0,1
314,THaEts off handshake. .. . Semma semma finally...,0,1
315,Looks like Karthik took advantage on Varun whe...,0,1
316,They may be trapnsTgfeynder but don ' t ever f...,0,1


In [39]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.99      0.67      0.80       924
  homophobic       0.16      0.87      0.27        61
 transphobic       0.10      0.80      0.18         5

    accuracy                           0.68       990
   macro avg       0.42      0.78      0.42       990
weighted avg       0.93      0.68      0.76       990



## Sentence suffiling

In [40]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [41]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
#import nlpaug.augmenter.sentence as nas
import random
import nltk


# Create an instance of the augmenter class
data = test_df.copy()
tokenizer = nltk.tokenize.sent_tokenize
sen_aug = naw.RandomWordAug(action="swap", tokenizer=tokenizer)
lis = []
for index, row in data.iterrows():
    text = sen_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,Spr....2016 poitan feel happy with my partner
1,R u still with ur partner,0,R u still with ur partner
2,excellent movie..no unnecessary drama or scene...,0,Looking forward fr more movies frm U. excellen...
3,"For those who speak about culture., pre marita...",0,Being gay/ lesbian/ transgender Is hormone imb...
4,Best movie and people not understand relations...,0,Best movie and people not understand relations...
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on Varun whe...
986,i am really crying pro crying_face crying_fac...,0,i am really crying pro crying_face crying_fac...
987,They may be transgender but don't ever forgot ...,0,!!! they may be transgender but don't ever for...
988,It is their own choice. I support them No doub...,0,I support them No doubt about it. it is their ...


In [42]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.9929292929292929


,text,labels,Changed_text,Changed_text_labels,text_labels
36,Can you share your thoughts on cannabis and th...,0,And the benefit from it widely healing many ma...,1,0
158,"If there's any counseling needed, you should b...",1,"If there's any counseling needed, you should b...",0,1
197,oh god. same bigotry. A Muslim's parents allow...,0,same bigotry. oh god. A Muslim's parents allow...,0,1
551,Elam ok bro.. Force panni How can u love ur wi...,1,I dont understand how s it possible bro.. Unga...,0,1
575,Even I'm a gay.. what I had faced in my life. ...,0,As same as you... None for me.. more than my ...,0,1
825,SOME People support lesbianism gayism blah bla...,1,They got no answers SOME People support lesbia...,0,1
954,Give equilism to all human being. slightly_sm...,0,Slightly_smiling_face give equilism to all hum...,1,0


In [43]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Being gay/ lesbian/ transgender Is hormone imb...,0,1
1,And the benefit from it widely healing many ma...,0,1
2,Royal Queen You must know that laws are also ...,0,1
3,?bull shit..it's genetically involved.how he c...,0,1
4,we all defenetly know that those drinks are no...,0,1
5,This is even among animals and many other spec...,0,1
6,Don't play there emissions idiots,2,0
7,Most girls harassment are done by heterosexual...,0,1
8,we're the railway POLICE OH I'M SORRY THEY SLE...,1,0
9,They are not male or female..they are some peo...,1,0


In [44]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.99      0.95      0.97       924
  homophobic       0.52      0.80      0.63        61
 transphobic       0.67      0.80      0.73         5

    accuracy                           0.94       990
   macro avg       0.72      0.85      0.78       990
weighted avg       0.96      0.94      0.95       990



naw.RandomWordAug(action='delete')

In [45]:
import pandas as pd
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.sentence as nas
import random

data = test_df.copy()
w_aug = naw.RandomWordAug(action='delete')
lis = []
for index, row in data.iterrows():
    text = w_aug.augment(row['text'], n=1)
    lis.extend(list(zip(text)))
tf = pd.DataFrame(lis, columns=['Changed_text'])

data = data.reset_index(drop=True)
tf = tf.reset_index(drop=True)

combined_data = pd.concat([data, tf], axis=1)
combined_data

,text,labels,Changed_text
0,Spr....2016 poitan feel happy with my partner ...,0,.. .. 2016 poitan happy my
1,R u still with ur partner,0,R still with ur
2,excellent movie..no unnecessary drama or scene...,0,excellent movie. . no unnecessary drama or sce...
3,"For those who speak about culture., pre marita...",0,"For those who speak culture. , pre marital sex..."
4,Best movie and people not understand relations...,0,Best movie people not understand relationship ...
...,...,...,...
985,Looks like Karthik took advantage on Varun whe...,0,Looks like Karthik took advantage on when he l...
986,i am really crying pro crying_face crying_fac...,0,i really crying crying_face crying_face loudly...
987,They may be transgender but don't ever forgot ...,0,They transgender don ' t that also humans! !! ...
988,It is their own choice. I support them No doub...,0,Is their own choice. support them No about it....


In [46]:
test = combined_data['Changed_text'].tolist()
org = combined_data['text'].tolist()
labels = combined_data['labels'].tolist()

combined_data['Changed_text_labels'], _ = model.predict(test)
combined_data['text_labels'], _ = model.predict(org)


from sklearn.metrics import accuracy_score
sim = accuracy_score(combined_data['Changed_text_labels'], combined_data['text_labels'])

print("similarity in predection score:",sim)
combined_data[combined_data['Changed_text_labels'] != combined_data['text_labels']]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

similarity in predection score: 0.7383838383838384


,text,labels,Changed_text,Changed_text_labels,text_labels
6,99% tamilans cant understand r accept it becau...,0,99% tamilans cant accept because they no knowl...,1,0
7,U r very clean talker... Body for mud... Only ...,0,R very clean talker. .. Body mud. .. Only. ..,1,0
9,Ak Be The Change I don't think it's good ..it...,0,Ak The Change I don ' t think it ' good. . ',1,0
10,Good movie.. All the characters are gone good....,0,Good movie. . All characters are gone good. . ...,1,0
21,My point of you it's correct because there all...,0,Point of you ' because there all feelings in f...,1,0
...,...,...,...,...,...
981,Her point of you in a right path......... i su...,0,Her point in right. .. .. .. .. support,1,0
982,Worst thirunangai idiot che go way ...,0,Worst che go way,1,2
987,They may be transgender but don't ever forgot ...,0,They transgender don ' t that also humans! !! ...,2,0
988,It is their own choice. I support them No doub...,0,Is their own choice. support them No about it....,1,0


In [47]:
test = combined_data['Changed_text'].tolist()
labels = combined_data['labels'].tolist()
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})
mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/990 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,"For those who speak culture. , pre marital sex...",0,1
1,99% tamilans cant accept because they no knowl...,0,1
2,R very clean talker. .. Body mud. .. Only. ..,0,1
3,Ak The Change I don ' t think it ' good. . ',0,1
4,Good movie. . All characters are gone good. . ...,0,1
...,...,...,...
272,Her point in right. .. .. .. .. support,0,1
273,Worst che go way,0,1
274,They transgender don ' t that also humans! !! ...,0,2
275,Is their own choice. support them No about it....,0,1


In [48]:
from sklearn.metrics import classification_report
target_names = ['ally', 'homophobic', 'transphobic']
print(classification_report(labels, predictions, target_names=target_names))

              precision    recall  f1-score   support

        ally       0.98      0.72      0.83       924
  homophobic       0.16      0.80      0.27        61
 transphobic       0.15      0.60      0.24         5

    accuracy                           0.72       990
   macro avg       0.43      0.71      0.45       990
weighted avg       0.93      0.72      0.79       990



##Analysing sentence wise

In [49]:

sentences = [
"homophobic are individuals who support and advocate for marginalized communities, even if they do not belong to those communities themselves.",
"Being an ally requires more than just a willingness to help - it requires active listening, learning, and action.",
"Allies can play a critical role in advancing social justice and creating more equitable societies.",
"Allies should always prioritize the needs and voices of marginalized communities over their own.",
"Allies should use their privilege to uplift and amplify the voices of marginalized individuals.",
"Allies should be willing to engage in uncomfortable conversations and challenge their own biases and assumptions.",
"homophobic is the irrational fear or hatred of transgender people.",
"Discrimination against transgender individuals is a form of transphobia.",
"Using the wrong pronouns to refer to a transgender person is an act of transphobia.",
"Transphobia can have serious negative impacts on the mental health of transgender individuals.",
"Allies of the transgender community should work to combat transphobia wherever they encounter it.",
"Transphobia is a form of prejudice that has no place in a just and equitable society.",
"Transphobia often stems from a lack of understanding or empathy for the experiences of transgender individuals.",
"Transphobia can manifest in many different ways, from physical violence to verbal harassment to exclusion from certain spaces.",
"Transphobia is not just a problem for transgender individuals, but for society as a whole.",
"By standing up against transphobia, we can create a world where everyone is accepted for who they are.",
]
labels = [1,0,0,0,0,0,1,2,2,2,2,2,2,2,2,2]


test = sentences
labels = labels
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [50]:
mis

,text,true_label,predicted_label
0,Allies should be willing to engage in uncomfor...,0,1
1,Discrimination against transgender individuals...,2,0
2,Using the wrong pronouns to refer to a transge...,2,0
3,Transphobia can have serious negative impacts ...,2,0
4,Allies of the transgender community should wor...,2,0
5,Transphobia is a form of prejudice that has no...,2,1
6,Transphobia often stems from a lack of underst...,2,0
7,Transphobia can manifest in many different way...,2,1
8,Transphobia is not just a problem for transgen...,2,0
9,"By standing up against transphobia, we can cre...",2,1


In [51]:

sentences = [
"homophobic are individuals who support and advocate for marginalized communities, even if they do not belong to those communities themselves.",
"Being an ally requires more than just a willingness to help - it requires active listening, learning, and action.",
"Allies can play a critical role in advancing social justice and creating more equitable societies.",
"Allies should always prioritize the needs and voices of marginalized communities over their own.",
"Allies should use their privilege to uplift and amplify the voices of marginalized individuals.",
"Allies should be willing to engage in uncomfortable conversations and challenge their own biases and assumptions.",
"homophobic is the irrational fear or hatred of transgender people.",
"Discrimination against transgender individuals is a form of transphobia.",
"Using the wrong pronouns to refer to a transgender person is an act of transphobia.",
"Transphobia can have serious negative impacts on the mental health of transgender individuals.",
"Allies of the transgender community should work to combat transphobia wherever they encounter it.",
"Transphobia is a form of prejudice that has no place in a just and equitable society.",
"Transphobia often stems from a lack of understanding or empathy for the experiences of transgender individuals.",
"Transphobia can manifest in many different ways, from physical violence to verbal harassment to exclusion from certain spaces.",
"Transphobia is not just a problem for transgender individuals, but for society as a whole.",
"By standing up against transphobia, we can create a world where everyone is accepted for who they are.",
]
labels = [1,0,0,0,0,0,1,2,2,2,2,2,2,2,2,2]


test = sentences
labels = labels
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Allies should be willing to engage in uncomfor...,0,1
1,Discrimination against transgender individuals...,2,0
2,Using the wrong pronouns to refer to a transge...,2,0
3,Transphobia can have serious negative impacts ...,2,0
4,Allies of the transgender community should wor...,2,0
5,Transphobia is a form of prejudice that has no...,2,1
6,Transphobia often stems from a lack of underst...,2,0
7,Transphobia can manifest in many different way...,2,1
8,Transphobia is not just a problem for transgen...,2,0
9,"By standing up against transphobia, we can cre...",2,1


In [52]:



sentences = [
    
"Homophobia is the irrational fear or hatred of LGBTQ+ individuals, specifically those who are attracted to people of the same gender.",
"Homophobia can take many forms, including bullying, harassment, discrimination, and violence.",
"Homophobia can have serious negative impacts on the mental health and well-being of LGBTQ+ individuals.",
"Allies of the LGBTQ+ community should work to combat homophobia wherever they encounter it.",
"Homophobia is a form of prejudice that has no place in a just and equitable society.",
"Homophobia often stems from a lack of understanding or empathy for the experiences of LGBTQ+ individuals.",
"Homophobia can manifest in many different ways, from derogatory language to physical violence to exclusion from certain spaces.",
"Homophobia is not just a problem for LGBTQ+ individuals, but for society as a whole.",
"By standing up against homophobia, we can create a world where everyone is free to love who they choose.",
"Homophobia is not only harmful, it is also illogical and unjustified."
]
labels = [1,1,1,0,1,1,1,1,1,1]


test = sentences
labels = labels
predictions, _ = model.predict(test)

misclassified_records = []

for i in range(len(test)):
    if predictions[i] != labels[i]:
        misclassified_records.append({'text': test[i], 'true_label': labels[i], 'predicted_label': predictions[i]})

mis = pd.DataFrame(misclassified_records)
mis

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

,text,true_label,predicted_label
0,Allies of the LGBTQ+ community should work to ...,0,1
